# Backtesting ML Regression-Based

In [ ]:
???

Classification Strategy | Regression Strategy
-|-
![](src/res_classification.png) | ![](src/res_regression.png)

## Load the model

In [1]:
import pickle

In [2]:
with open('models/model_dt_regression.pkl', 'rb') as f:
    model_dt = pickle.load(f)

In [3]:
model_dt

DecisionTreeRegressor(max_depth=15)

## Load the data

In [4]:
import pandas as pd

df = pd.read_excel('data/INTC_subset_Processed.xlsx', index_col=0, parse_dates=['Date'])
df

,Open,High,Low,Close,Volume,change_tomorrow,change_tomorrow_direction
Date,,,,,,,
2020-01-02,60.240002,60.970001,60.220001,60.840000,18056000,-1.231284,DOWN
2020-01-03,59.810001,60.700001,59.810001,60.099998,15293900,-0.283661,DOWN
2020-01-06,59.590000,60.200001,59.330002,59.930000,17755200,-1.696929,DOWN
2020-01-07,59.779999,59.799999,58.889999,58.930000,21876100,0.067833,UP
2020-01-08,58.889999,59.320000,58.520000,58.970001,23133500,0.556489,UP
...,...,...,...,...,...,...,...
2024-09-26,24.280001,24.420000,23.250000,23.920000,95416900,-0.041824,DOWN
2024-09-27,24.160000,24.660000,23.700001,23.910000,85883300,-1.918162,DOWN
2024-09-30,23.740000,23.950001,23.090000,23.459999,66308200,-3.393559,DOWN


## Backtesting.py Library

### Create your Strategy Class

In [5]:
from backtesting import Backtest, Strategy

In [6]:
class Regression(Strategy):
    def init(self):
        self.model = model_dt
        self.already_bought = False

    def next(self):
        explanatory_today = self.data.df.iloc[[-1], :]
        forecast_tomorrow = self.model.predict(explanatory_today)[0]
        
        if forecast_tomorrow > 1 and self.already_bought == False:
            self.buy()
            self.already_bought = True
        elif forecast_tomorrow < -5 and self.already_bought == True:
            self.sell()
            self.already_bought = False
        else:
            pass

### Define initial conditions

In [7]:
df_explanatory = df[['Open', 'High', 'Low', 'Close', 'Volume']].copy()

In [8]:
df_explanatory.iloc[-1:, :]

,Open,High,Low,Close,Volume
Date,,,,,
2024-10-02,22.67,23.07,22.35,22.389999,59033900


In [9]:
bt = Backtest(df_explanatory, Regression,
              cash=10000, commission=.002, exclusive_orders=True)

### Run backtesting

In [10]:
results = bt.run()

### Interpret backtesting results

In [11]:
results.to_frame(name='Values').loc[:'Return [%]']

,Values
Start,2020-01-02 00:00:00
End,2024-10-02 00:00:00
Duration,1735 days 00:00:00
Exposure Time [%],98.913043
Equity Final [$],47657.826326
Equity Peak [$],50285.326326
Return [%],376.578263


### Visualize the strategy performance

In [12]:
bt.plot(filename='reports_backtesting/backtesting_regression.html')

c:\Users\richa\anaconda3\envs\algo_trading\Lib\site-packages\backtesting\_plotting.py:463: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  .resample(resample_rule, label='left')


GridPlot(id='p1318', ...)